In [3]:
!pip install scikit-dimension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm
import math
import re
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import os
from typing import List
import traceback

from transformers import AutoTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoModelForCausalLM, AutoTokenizer


import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_english = set(stopwords.words('english'))
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('sentiwordnet')
from skdim.id import MLE


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


In [5]:
# pip freeze

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Define device

In [ ]:
#BART Score Calculation

class BARTScorer:
    def __init__(self, device='cuda:0', max_length=1024, checkpoint='facebook/bart-base'):
        # Set up model
        self.device = device
        self.max_length = max_length
        self.tokenizer = BartTokenizer.from_pretrained(checkpoint)
        self.model = BartForConditionalGeneration.from_pretrained(checkpoint)
        self.model.eval()
        self.model.to(device)

        # Set up loss
        self.loss_fct = nn.NLLLoss(reduction='none', ignore_index=self.model.config.pad_token_id)
        self.lsm = nn.LogSoftmax(dim=1)

    def load(self, path=None):
        """ Load model from paraphrase finetuning """
        if path is None:
            path = 'models/bart.pth'
        self.model.load_state_dict(torch.load(path, map_location=self.device))

    def score(self, srcs, tgts, batch_size=4):
        """ Score a batch of examples """
        score_list = []
        # print(len(srcs))
        for i in range(0, len(srcs), batch_size):
            src_list = srcs[i: i + batch_size]
            tgt_list = tgts[i: i + batch_size]
            # print(src_list)
            try:
                with torch.no_grad():
                    encoded_src = self.tokenizer(
                        src_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    encoded_tgt = self.tokenizer(
                        tgt_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    src_tokens = encoded_src['input_ids'].to(self.device)
                    src_mask = encoded_src['attention_mask'].to(self.device)

                    tgt_tokens = encoded_tgt['input_ids'].to(self.device)
                    tgt_mask = encoded_tgt['attention_mask']
                    tgt_len = tgt_mask.sum(dim=1).to(self.device)
                    # print(tgt_tokens)

                    output = self.model(
                        input_ids=src_tokens,
                        attention_mask=src_mask,
                        labels=tgt_tokens
                    )
                    logits = output.logits.view(-1, self.model.config.vocab_size)
                    # print(logits.shape)
                    loss = self.loss_fct(self.lsm(logits), tgt_tokens.view(-1))
                    loss = loss.view(tgt_tokens.shape[0], -1)
                    # print(loss, tgt_len)
                    loss = loss.sum(dim=1) / tgt_len
                    # print(loss)
                    curr_score_list = [-x.item() for x in loss]
                    score_list += curr_score_list

            except RuntimeError:
                traceback.print_exc()
                print(f'source: {src_list}')
                print(f'target: {tgt_list}')
                exit(0)
        return score_list


In [ ]:
#LogRank, Loglikelihood
#     log_likelihood_x
#     log_rank_x
#     log_likelihood_x_tilde
#     miu_tilde
#     sigma_tilde

def get_likelihood(logits, labels):
    assert logits.shape[0] == 1
    assert labels.shape[0] == 1
    labels = labels.unsqueeze(-1) if labels.ndim == logits.ndim - 1 else labels
    lprobs = torch.log_softmax(logits, dim=-1)
    log_likelihood = lprobs.gather(dim=-1, index=labels)
    return log_likelihood.mean(dim=1)

def get_logrank(logits, labels):
    assert logits.shape[0] == 1
    assert labels.shape[0] == 1

    # get rank of each label token in the model's likelihood ordering
    matches = (logits.argsort(-1, descending=True) == labels.unsqueeze(-1)).nonzero()
    assert matches.shape[1] == 3, f"Expected 3 dimensions in matches tensor, got {matches.shape}"

    ranks, timesteps = matches[:, -1], matches[:, -2]

    # make sure we got exactly one match for each timestep in the sequence
    assert (timesteps == torch.arange(len(timesteps)).to(timesteps.device)).all(), "Expected one match per timestep"

    ranks = ranks.float() + 1  # convert to 1-indexed rank
    ranks = torch.log(ranks)
    return -ranks.mean().item()


In [ ]:
pct = 0.015

def fill_and_mask(text,  pct = pct):
    tokens = text.split(' ')
    # print(len(tokens))
    n_spans = pct * len(tokens)
    n_spans = int(n_spans)
    # print(n_spans)
    repeated_random_numbers = np.random.choice(range(len(tokens)), size=n_spans)

    return repeated_random_numbers.tolist()

def apply_extracted_fills(texts, indices_list=[]):
    tokens = [x.split(' ') for x in texts]

    for idx, (text, indices) in enumerate(zip(tokens, indices_list)):
        for idx in indices:
            text[idx] = ""

    # join tokens back into text
    texts = [" ".join(x) for x in tokens]
    return texts


def perturb_texts_(texts, pct=pct):

    indices_list = [fill_and_mask(x, pct) for x in texts]
    perturbed_texts = apply_extracted_fills(texts, indices_list)

    return perturbed_texts


def perturb_texts(texts, pct=pct):
    outputs = []
    for i in tqdm.tqdm(range(0, len(texts), 50), desc="Applying perturbations"):
        outputs.extend(perturb_texts_(texts[i:i + 50], pct))
    return outputs
def get_samples(logits, labels):
    nsamples = 10000
    lprobs = torch.log_softmax(logits, dim=-1)
    distrib = torch.distributions.categorical.Categorical(logits=lprobs)
    samples_2 = distrib.sample([nsamples]).permute([1, 2, 0])
    return samples_2

def get_score(logits_score,labels,source_texts,perturbed_texts):
  log_likelihood_x = get_likelihood(logits_score, labels)
  log_rank_x = get_logrank(logits_score, labels)
  source_texts_list = [source_texts]*10
  values = bart_scorer.score(perturbed_texts,source_texts_list, batch_size=10)
  mean_values = np.mean(values)
  lrr_score = (log_likelihood_x.squeeze(-1).item() / log_rank_x) * math.pow(math.e, -mean_values)
  loglikelihood_score = log_likelihood_x.squeeze(-1).item() * math.pow(math.e, mean_values)
  bart_score  = -mean_values
  samples_2 = get_samples(logits_score, labels)
  log_likelihood_x_tilde = get_likelihood(logits_score, samples_2)
  miu_tilde = log_likelihood_x_tilde.mean(dim=-1)
  sigma_tilde = log_likelihood_x_tilde.std(dim=-1)
  fast_score = ((log_likelihood_x.squeeze(-1).item() - miu_tilde.item()) / (sigma_tilde.item())) * math.pow(math.e, mean_values)
  return lrr_score, loglikelihood_score, bart_score, fast_score

def generate_scores(original_text, tokenizer,model):
  perturbed_original_texts = perturb_texts([x for x in [original_text] for _ in range(10)])
  tokenized = tokenizer(original_text,truncation=True, return_tensors="pt", return_token_type_ids=False).to(device)
  labels = tokenized.input_ids[:, 1:]
  logits_score = model(**tokenized).logits[:, :-1]
  lrr_score, loglikelihood_score, bart_srore,fast_score = get_score(logits_score,labels,original_text,perturbed_original_texts)
  return [lrr_score, loglikelihood_score,  bart_srore,fast_score]




In [ ]:
#POS tage
def find_pos_tags_spacy(sentence):
    doc = nlp(sentence)
    verbs = [token for token in doc if token.tag_ in ['VB','VBD', 'VBG', 'VBN', 'VBP','VBZ']]

    # Filter nouns, pronouns, and verbs
    proper_nouns = [token for token in doc if token.tag_ in [ 'NNP', 'NNPS']]
    nouns = [token for token in doc if token.tag_ in ['NNP', 'NNPS','NN', 'NNS' ]] # Nouns

    past_tense = [token for token in doc if token.tag_ in [ 'VBD', 'VBN']]
    intrg_words = [token for token in doc if token.tag_ in ['WRB', 'WDT', 'WP']]

    return len(nouns),len(past_tense),len(verbs)


def remove_stopwords(text, stopword_list):
    filtered_tokens = ' '.join(word.lower() for word in text.split() if word.lower() not in stopword_list)
    # print(filtered_tokens)
    return filtered_tokens

def stopwords_count(text, stopword_list):
    stop_tokens = [token for token in text.split() if token.lower() in stopword_list]
    return len(stop_tokens)

def remove_punctuation(text):
    text = re.sub(r'[\.]', ' ', text)
    text =  text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    return text
def out_of_vocab_count(text):
  filtered = remove_stopwords(text,stopwords_english)
  words = re.findall(r'\w+', filtered.lower())
  oov_words = [word for word in words if not list(swn.senti_synsets(word))]
  return len(oov_words)

def find_word_stats(text):
  clean_txt = remove_punctuation(text)
  words = clean_txt.split()
  len_words = len(words)
  ttr = len(set(words))/len(words)
  stopwords = stopwords_count(clean_txt, stopwords_english)
  out_of_vocab= out_of_vocab_count(clean_txt)
  return len_words, ttr,stopwords,out_of_vocab



In [ ]:


MLE_solver = MLE()
tokenizer_path = model_path = "roberta-base"
tokenizer_rob = RobertaTokenizer.from_pretrained(tokenizer_path)
model_rob = RobertaModel.from_pretrained(model_path)
model_rob.to(device)

def preprocess_text(text):
    return text.replace('\n', ' ').replace('  ', ' ')

def get_mle_single(text, solver):
    inputs = tokenizer_rob(preprocess_text(text), truncation=True, max_length=512, return_tensors="pt").to(device) # Move inputs to device
    with torch.no_grad():
        outp = model_rob(**inputs)

    return solver.fit_transform(outp[0][0].cpu().numpy()[1:-1]) # Move output back to CPU if needed

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfid_vectors(df):
    tfidf_vectorizer = TfidfVectorizer(max_features=200)
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
    return tfidf_matrix.toarray()

In [ ]:
def get_features(df,tokenizer,model):
  length_feature = np.array([[len(word_tokenize(t))] for t in df['text']])
  pos_tags = np.array([find_pos_tags_spacy(t) for t in df['text']])
  word_stats = np.array([find_word_stats(t) for t in df['text']])
  mle_score = np.array([[get_mle_single(t,MLE_solver)] for t in df['text']])
  tocsin = np.array([generate_scores(t,tokenizer,model) for t in df['text']])
  tfid_vals = tfid_vectors(df)
  return length_feature,pos_tags,word_stats,mle_score,tocsin,tfid_vals


In [ ]:
bart_scorer = BARTScorer(device=device, checkpoint='facebook/bart-base')


In [ ]:
train_df = pd.read_json("train.jsonl",lines=True)
val_df = pd.read_json("val.jsonl", lines=True)

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Makes CUDA errors synchronous for easier debugging
os.environ['TORCH_USE_CUDA_DSA'] = "1"  # Enables device-side assertions for more detailed error messages

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

print(device)
model.to(device)



cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# train_df=train_df.head(20)
# val_df = val_df.head(10)

In [ ]:
length_feature,pos_tags,word_stats,mle_score,tocsin,tfid = get_features(train_df,tokenizer,model)

Applying perturbations: 100%|██████████| 1/1 [00:00<00:00, 746.85it/s]


In [ ]:
X_features = np.hstack([
    length_feature,
    pos_tags,
    word_stats,
    mle_score,
    tocsin,
    tfid
])

y_train = train_df['label'].values

In [ ]:
X_features.shape

(20, 213)

In [8]:
# train_df = pd.read_json("train.jsonl",lines=True)
# val_df = pd.read_json("val.jsonl", lines=True)
# X_features = pd.read_csv('train_score_df.csv')
# train_pos = pd.read_csv('pos_stats.csv')
# X_features = pd.concat([X_features,train_pos],axis=1)
# X_features.drop(columns=['Unnamed: 0'],inplace=True)
# X_features.drop(columns=['intrg_words'],inplace=True)
# X_features.drop(columns=['proper_nouns'],inplace=True)
# X_features.head()
# X_features.shape



(23707, 213)

In [ ]:
# train_df.shape

(23707, 5)

In [9]:

import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
xgbclassifier = xgb.XGBClassifier(use_label_encoder=False,
                    eval_metric='logloss',random_state=42)
y_train = train_df['label']
xgbclassifier.fit(X_features, y_train)



eval_set = [(X_features, y_train)]

# Fit the model, providing the evaluation set
xgbclassifier.fit(
    X_features,
    y_train,
    eval_set=eval_set,
    verbose=False # Set to True or a number to see progress during training
)

# Retrieve the evaluation results
results = xgbclassifier.evals_result()
train_logloss = results['validation_0']['logloss'] # 'validation_0' corresponds to the first item in eval_set

# Plot the training loss curve for XGBoost
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_logloss) + 1), train_logloss, label='XGBoost Training LogLoss')
plt.title('XGBoost Training Loss Curve')
plt.xlabel('Boosting Rounds (Iterations)')
plt.ylabel('LogLoss')
plt.legend()
plt.grid(True)
plt.show()

print("XGBoost training complete.")

# Get predictions from XGBoost (as input for the meta-model)
# Use predict_proba as you were doing, good for stacking
final_preds = xgbclassifier.predict(X_features)
xgbclassifier.save_model("tocsin_id.json")


XGBoost training complete.


In [10]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train, final_preds)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [ ]:
length_feature,pos_tags,word_stats,mle_score,tocsin,tfid = get_features(val_df,tokenizer,model)

In [ ]:
X_features = np.hstack([
    length_feature,
    pos_tags,
    word_stats,
    mle_score,
    tocsin,
    tfid
])

y_val = val_df['label']

In [13]:
# model_xgb_2 = xgb.XGBClassifier(use_label_encoder=False,
#                     eval_metric='logloss',random_state=42)
# model_xgb_2.load_model("tocsin_id.json")
# X_features = pd.read_csv('val_score_df.csv')
# val_pos = pd.read_csv('pos_val_stats.csv')
# X_features = pd.concat([X_features,val_pos],axis=1)
# X_features.drop(columns=['Unnamed: 0'],inplace=True)
# X_features.drop(columns=['intrg_words'],inplace=True)
# X_features.drop(columns=['proper_nouns'],inplace=True)
# X_features.head()
# y_val = val_df['label']

In [14]:
from sklearn.metrics import classification_report

final_preds = xgbclassifier.predict(X_features)
# final_preds = rf.predict(xgb_test_preds)
print(classification_report(y_val, final_preds, target_names=["Human", "AI"]))

accuracy = accuracy_score(y_val, final_preds)
print(f"Accuracy: {accuracy}")

              precision    recall  f1-score   support

       Human       0.98      0.89      0.94      1277
          AI       0.94      0.99      0.97      2312

    accuracy                           0.96      3589
   macro avg       0.96      0.94      0.95      3589
weighted avg       0.96      0.96      0.96      3589

Accuracy: 0.9573697408748956


In [15]:
from sklearn.metrics import f1_score

# Assuming 'y_val' are the true labels and 'final_preds' are the predicted labels
f1 = f1_score(y_val, final_preds)
print(f"F1 Score: {f1}")

F1 Score: 0.9677283273570977
